<a href="https://colab.research.google.com/github/hecaguayo/Tarea1MLFBI/blob/master/Tarea_1_MLBI_copia1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Tarea 1 de MLBI 2020-1**
Hector Aguayo & Fabián Segura

# IMPORTACION DE MODULOS REQUERIDOS

Se importan numpy, pandas, tensorflow, etc

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import logging
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger("tensorflow").setLevel(logging.WARNING)

import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(2)

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.2.0-rc3


Se deben leer los datos de los dataset envíados, el primero es un archivo .xls

# LECTURA DE DATOS

Se abre archivo desde Github y se importan usando pandas

In [0]:
url = 'https://github.com/FxknTT/TareaMLBI1/blob/master/dataset1.xlsx?raw=true'
dataset = pd.read_excel(url)

Se verifica que ningun dato falte

In [3]:
dataset.isna().sum()

crr_Nom                     0
psu_Nem                     0
psu_Leng                    0
psu_Mate                    0
psu_Cie                     0
psu_Pond                    0
Longitud                    0
Latitud                     0
cred_apr                    0
distancia                   0
F                           0
M                           0
PARTICULAR_SUBVENCIONADO    0
PARTICULAR_PAGADO           0
MUNICIPAL                   0
cluster                     0
dtype: int64

# LIMPIEZA DE DATOS

Se debe alertar que "crr_Nom" es categorica, no numerica. Se convierte a un "one hot"

Cambiando las carrearas a filas one hot (variables dummy)

In [0]:
carreras = pd.get_dummies(dataset['crr_Nom'],prefix="Carrera")

Ahora obtenemos una nueva tabla llamada carrera en donde nos indica con valores 0 o 1 si pertenece o no a alguna Ingeniería.

In [5]:
carreras.head() 

,Carrera_INGENIERIA CIVIL,Carrera_INGENIERIA CIVIL - PLAN COMUN,Carrera_INGENIERIA CIVIL AEROESPACIAL,Carrera_INGENIERIA CIVIL BIOMEDICA,Carrera_INGENIERIA CIVIL DE MATERIALES,Carrera_INGENIERIA CIVIL ELECTRICA,Carrera_INGENIERIA CIVIL ELECTRONICA,Carrera_INGENIERIA CIVIL EN TELECOMUNICACIONES,Carrera_INGENIERIA CIVIL INDUSTRIAL,Carrera_INGENIERIA CIVIL INFORMATICA,Carrera_INGENIERIA CIVIL MECANICA,Carrera_INGENIERIA CIVIL METALURGICA,Carrera_INGENIERIA CIVIL QUIMICA,Carrera_INGENIERÍA CIVIL DE MINAS
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Ahora juntamos nuestra dataset original con la nueva tabla obtenida.

In [0]:
dataset = pd.concat([dataset, carreras], axis=1)

In [7]:
dataset.head()

,crr_Nom,psu_Nem,psu_Leng,psu_Mate,psu_Cie,psu_Pond,Longitud,Latitud,cred_apr,distancia,F,M,PARTICULAR_SUBVENCIONADO,PARTICULAR_PAGADO,MUNICIPAL,cluster,Carrera_INGENIERIA CIVIL,Carrera_INGENIERIA CIVIL - PLAN COMUN,Carrera_INGENIERIA CIVIL AEROESPACIAL,Carrera_INGENIERIA CIVIL BIOMEDICA,Carrera_INGENIERIA CIVIL DE MATERIALES,Carrera_INGENIERIA CIVIL ELECTRICA,Carrera_INGENIERIA CIVIL ELECTRONICA,Carrera_INGENIERIA CIVIL EN TELECOMUNICACIONES,Carrera_INGENIERIA CIVIL INDUSTRIAL,Carrera_INGENIERIA CIVIL INFORMATICA,Carrera_INGENIERIA CIVIL MECANICA,Carrera_INGENIERIA CIVIL METALURGICA,Carrera_INGENIERIA CIVIL QUIMICA,Carrera_INGENIERÍA CIVIL DE MINAS
0,INGENIERIA CIVIL INDUSTRIAL,783,653,685,672,735,-73.16487,-36.978759,38,20.201244,1,0,1,0,0,C2,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,INGENIERIA CIVIL BIOMEDICA,799,705,713,686,756,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,INGENIERIA CIVIL QUIMICA,688,543,612,534,638,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C3,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,INGENIERIA CIVIL ELECTRICA,605,598,658,601,633,-73.16487,-36.978759,21,20.201244,0,1,1,0,0,C2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,INGENIERIA CIVIL INFORMATICA,603,710,668,651,657,-73.16487,-36.978759,19,20.201244,0,1,1,0,0,C2,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Ahora procedemos a quitar la variale Crr_Nom

In [0]:
dataset = dataset.drop(["crr_Nom"],axis=1)

Revisamos nuestras variables

In [9]:
dataset.isna().sum() #Revisamos que todas las variables tengan datos.

psu_Nem                                           0
psu_Leng                                          0
psu_Mate                                          0
psu_Cie                                           0
psu_Pond                                          0
Longitud                                          0
Latitud                                           0
cred_apr                                          0
distancia                                         0
F                                                 0
M                                                 0
PARTICULAR_SUBVENCIONADO                          0
PARTICULAR_PAGADO                                 0
MUNICIPAL                                         0
cluster                                           0
Carrera_INGENIERIA CIVIL                          0
Carrera_INGENIERIA CIVIL - PLAN COMUN             0
Carrera_INGENIERIA CIVIL AEROESPACIAL             0
Carrera_INGENIERIA CIVIL BIOMEDICA                0
Carrera_INGE

**Ahora procedemos a realizar los mismos pasos anteriores con la variables CLUSTER.**

In [10]:
dataset.head()

,psu_Nem,psu_Leng,psu_Mate,psu_Cie,psu_Pond,Longitud,Latitud,cred_apr,distancia,F,M,PARTICULAR_SUBVENCIONADO,PARTICULAR_PAGADO,MUNICIPAL,cluster,Carrera_INGENIERIA CIVIL,Carrera_INGENIERIA CIVIL - PLAN COMUN,Carrera_INGENIERIA CIVIL AEROESPACIAL,Carrera_INGENIERIA CIVIL BIOMEDICA,Carrera_INGENIERIA CIVIL DE MATERIALES,Carrera_INGENIERIA CIVIL ELECTRICA,Carrera_INGENIERIA CIVIL ELECTRONICA,Carrera_INGENIERIA CIVIL EN TELECOMUNICACIONES,Carrera_INGENIERIA CIVIL INDUSTRIAL,Carrera_INGENIERIA CIVIL INFORMATICA,Carrera_INGENIERIA CIVIL MECANICA,Carrera_INGENIERIA CIVIL METALURGICA,Carrera_INGENIERIA CIVIL QUIMICA,Carrera_INGENIERÍA CIVIL DE MINAS
0,783,653,685,672,735,-73.16487,-36.978759,38,20.201244,1,0,1,0,0,C2,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,799,705,713,686,756,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,688,543,612,534,638,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C3,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,605,598,658,601,633,-73.16487,-36.978759,21,20.201244,0,1,1,0,0,C2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,603,710,668,651,657,-73.16487,-36.978759,19,20.201244,0,1,1,0,0,C2,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Creamos una variable y, que tiene los valores que deseamos predecir.

In [11]:
y = pd.get_dummies(dataset['cluster'],prefix="CLUSTER")
print(y)

      CLUSTER_C1  CLUSTER_C2  CLUSTER_C3  CLUSTER_C4
0              0           1           0           0
1              1           0           0           0
2              0           0           1           0
3              0           1           0           0
4              0           1           0           0
...          ...         ...         ...         ...
7861           0           1           0           0
7862           0           1           0           0
7863           0           0           1           0
7864           0           1           0           0
7865           0           0           0           1

[7866 rows x 4 columns]


Por último quitamos la variable cluster de nuestro dataset.



In [0]:
dataset = dataset.drop(["cluster"],axis=1)

In [13]:
print(dataset.isna().sum())
print(y.isna().sum())

psu_Nem                                           0
psu_Leng                                          0
psu_Mate                                          0
psu_Cie                                           0
psu_Pond                                          0
Longitud                                          0
Latitud                                           0
cred_apr                                          0
distancia                                         0
F                                                 0
M                                                 0
PARTICULAR_SUBVENCIONADO                          0
PARTICULAR_PAGADO                                 0
MUNICIPAL                                         0
Carrera_INGENIERIA CIVIL                          0
Carrera_INGENIERIA CIVIL - PLAN COMUN             0
Carrera_INGENIERIA CIVIL AEROESPACIAL             0
Carrera_INGENIERIA CIVIL BIOMEDICA                0
Carrera_INGENIERIA CIVIL DE MATERIALES            0
Carrera_INGE

Finalmente tenemos nuestro dataset con sólo variables numericas y además nuestra matriz y con los valores que desaemos predecir.

# DEFINIENDO RANGOS DE ENTRENAMIENTO Y TESTEO

Primero realizamos un cast a nuestro dataset y nuestra tabla y, transformando ambas en un dtype=tf.float64

In [0]:
x = tf.cast(dataset, dtype=tf.float64)
y = tf.cast(y, dtype=tf.float64)

Revisamos que se haya realizado correctamente

In [15]:
print(x)
print(y)

tf.Tensor(
[[783. 653. 685. ...   0.   0.   0.]
 [799. 705. 713. ...   0.   0.   0.]
 [688. 543. 612. ...   0.   1.   0.]
 ...
 [641. 598. 640. ...   0.   0.   0.]
 [641. 653. 595. ...   0.   0.   0.]
 [641. 524. 554. ...   0.   0.   0.]], shape=(7866, 28), dtype=float64)
tf.Tensor(
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]], shape=(7866, 4), dtype=float64)


Ahora dividimos nuestros datos en un 80% den entrenamiento y un 20% en testeo.

In [0]:
data_size = len(dataset)
train_pct = 0.8

train_size = int(data_size * train_pct)
print(train_size)

x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

6292


In [0]:
#x_train.shape[0], x_test.shape[0]
#x_train

Revisamos

In [0]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

tf.Tensor(
[[783. 653. 685. ...   0.   0.   0.]
 [799. 705. 713. ...   0.   0.   0.]
 [688. 543. 612. ...   0.   1.   0.]
 ...
 [661. 626. 759. ...   0.   0.   0.]
 [661. 620. 708. ...   0.   0.   0.]
 [599. 547. 694. ...   0.   0.   0.]], shape=(6292, 28), dtype=float64)
tf.Tensor(
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]], shape=(6292, 4), dtype=float64)
tf.Tensor(
[[661. 542. 637. ...   1.   0.   0.]
 [608. 560. 648. ...   1.   0.   0.]
 [554. 660. 664. ...   1.   0.   0.]
 ...
 [641. 598. 640. ...   0.   0.   0.]
 [641. 653. 595. ...   0.   0.   0.]
 [641. 524. 554. ...   0.   0.   0.]], shape=(1574, 28), dtype=float64)
tf.Tensor(
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]], shape=(1574, 4), dtype=float64)


# CREACION DE MODELO

Definición de layers

Modelo

Acá es para usar lo de tensorboard, pero no sé como funciona en Colab

In [0]:
logdir = "logs\\tf_keras\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


TypeError: ignored

El modelo es de un solo layery es lineal

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, activation = 'softmax', input_shape=(28,))
])
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer=keras.optimizers.Adam()
)

Sumario para ver las variables

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4)                 116       
Total params: 116
Trainable params: 116
Non-trainable params: 0
_________________________________________________________________


Se entrenan las variables

In [0]:
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size = train_size,
    verbose = 1, # Suppress chatty output; use Tensorboard instead
    epochs = 100,
    validation_data = (x_test, y_test),
    callbacks=[tensorboard_callback]
)

Epoch 1/100


ValueError: ignored

Ver como se entrenaron los pesos

In [0]:
print("Average test loss: ", np.average(training_history.history['loss']))

print('Prediction: {}'.format(model.predict(x_test)))
predictions = model.predict(x_test)

Average test loss:  0.38059904098510744
Prediction: [[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [0]:
capas = model.layers[0]
x,a = capas.get_weights()
print(x)
print(a)

[[ 8.46354440e-02 -2.71371305e-01 -2.02702776e-01 -4.00564730e-01]
 [-2.72714138e-01 -3.76971036e-01  3.38143319e-01  1.44136950e-01]
 [ 4.45740484e-02  3.90085369e-01  5.16170442e-01  2.26284862e-01]
 [ 4.36927453e-02  4.26862240e-02 -3.98655655e-03  1.27980933e-01]
 [ 3.32012236e-01  2.51624078e-01  4.12439615e-01 -1.83718815e-01]
 [-5.27115501e-02  1.19571418e-01 -1.63915187e-01  1.50052428e-01]
 [-3.12995583e-01 -9.00611579e-02 -4.09032255e-01  1.38216779e-01]
 [ 4.19725001e-01  3.11164469e-01 -4.26197529e-01  6.30709380e-02]
 [ 1.38997674e-01 -2.39277720e-01  1.51062340e-01 -3.18127513e-01]
 [ 1.96630433e-01  2.36576885e-01  3.95491153e-01 -2.14997545e-01]
 [-1.41749933e-01 -2.87392020e-01 -3.63212198e-01  1.17941365e-01]
 [ 1.36106208e-01 -1.17069095e-01 -3.19279820e-01  4.07732546e-01]
 [ 3.30953628e-01  3.25281888e-01  2.61489779e-01  1.94258377e-01]
 [-2.08377346e-01 -9.04378295e-03  8.50802511e-02  2.01541126e-01]
 [ 1.19697660e-01 -9.98768210e-02  1.14689499e-01  4.18892652e